In [1]:
import pymongo
import textacy
from bs4 import BeautifulSoup as bs

In [2]:
client = pymongo.MongoClient()

In [3]:
db = client.BlogData

In [4]:
coll = db.PhysOrg

In [5]:
coll.count()

23501

In [6]:
textacy_preprocessor = lambda text: textacy.preprocess.preprocess_text(text,
                                                                       no_contractions=True,
                                                                       no_numbers=True,
                                                                       no_emails=True,
                                                                       no_currency_symbols=True,
                                                                       lowercase=True)

In [7]:
import sys

In [8]:
sys.path.append('/home/immersinn/gits/weblog_analytics/src/')

In [9]:
import article_handles

In [10]:
# Init the iterator over articles from MDB
mdb_art_iter = coll.find({'html' : {'$ne' : 'None'}, 'url' : {'$regex' : '^http://phys.org/news/'}}).limit(20)

In [11]:
from itertools import tee, starmap
from cytoolz.itertoolz import cons, pluck

In [12]:
def unzip(seq):
    """
    Borrowed from ``toolz.sandbox.core.unzip``, but using cytoolz instead of toolz
    to avoid the additional dependency.
    """
    seq = iter(seq)
    # check how many iterators we need
    try:
        first = tuple(next(seq))
    except StopIteration:
        return tuple()
    # and create them
    niters = len(first)
    seqs = tee(cons(first, seq), niters)
    return tuple(starmap(pluck, enumerate(seqs)))

In [15]:
def proc_art_iterator(mdb_iterator, tpp):
    def proc_item(entry):
    ##    # Step 01: Get next doc from MongoDB
    ##    entry = mdb_iterator.next()

        # Step 02: Extract Content from Article
        article = article_handles.articleObjectPhysOrg(entry)
        article.processArticle()

        # Step 03: pre-process Content with textacy
        content = tpp(article.contents)
        
        return(content, article.meta)

    return unzip((proc_item(item) for item in mdb_iterator))

In [16]:
content_stream, metadata_stream = proc_art_iterator(mdb_art_iter, textacy_preprocessor)

In [17]:
type(content_stream)

cytoolz.itertoolz._pluck_index

In [18]:
corpus = textacy.Corpus("en", texts=content_stream, metadatas=metadata_stream)

In [19]:
corpus.n_docs

19

In [20]:
doc = corpus[0]

In [21]:
doc.corpus_index

0

In [22]:
import numpy

In [24]:
perm = numpy.random.permutation(range(20))

In [25]:
remove = perm[:15]

In [26]:
corpus.remove(lambda doc: doc.corpus_index in remove)

In [27]:
corpus.n_docs

5